# New England Shelf Basin Bathymetry

**Purpose:** store regional bathymetry into netCDF

Description: Save bathymetry of area that includes the [OOI Coastal Pioneer profiler moorings](https://ooinet.oceanobservatories.org), and that is bounded by land on the west and north. The data is bounded in longitude, $[75.5 \:\: 70]^\circ $W, and in latitude, $[38 \:\: 42]^\circ $N.

Reference: Tozer, Brook, et al. "Global bathymetry and topography at 15 arc sec: SRTM15+." _Earth and Space Science 6.10_ (2019): 1847-1864. https://topex.ucsd.edu/sandwell/publications/180_Tozer_SRTM15+.pdf

Data from: https://topex.ucsd.edu/cgi-bin/get_data.cgi

In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import os

In [2]:
# specify downloaded data file
if os.getenv('HOSTNAME') is not None:
    bathyDir = '/scratch2/ivana/regionalgcm/data'
else: 
    bathyDir = '/Users/ivanaescobar/work/pioneer'
fBathyHighRes = f'{bathyDir}/nesbaHighResBathymetry.csv'
fBathy = f'{bathyDir}/nesbaBathymetry.csv'
bathyNames = ['lon','lat','depth']

bathyHighRes = pd.read_csv(fBathyHighRes,sep='\t',names=bathyNames,
                      header=None,comment='%')
bathyDf = pd.read_csv(fBathy,sep=',',names=bathyNames,
                      header=None,comment='%')

## Transform into xarray.Dataset

In [3]:
def getRegionDims (df): 
    """
    Input: 
        df: pandas.DataFrame
    Returns:
        j: length of lat dimension
        i: length of lon dimension
    Note: WORKS BECAUSE RAW DATA IS PERIODIC
    """
    j = len( np.unique(df['lat']) )
    i = len( np.unique(df['lon']) )
    print('Unique values:\nlat: %i\nlon: %i\ncheck for completeness: %i' \
          %(j, i, j*i-len(df['depth'])) )
    return i,j

def lons360to180 (arr):
    return ((arr+180)%360)-180

In [4]:
def make_ij_dataset (df):
    
    # get dimensions of regional space
    iLen,jLen = getRegionDims(df)
    i=np.arange(iLen)
    j=np.arange(jLen)
    
    # write data in i, j array shapes
    ds=xr.Dataset()
    ds['i']   = xr.DataArray(i,coords={'i':i},dims=('i',))
    ds['j']   = xr.DataArray(j,coords={'j':j},dims=('j',))
    ds['lon'] = xr.DataArray(np.reshape( df['lon'].values, (jLen, iLen) ),\
                             coords={'j':j,'i':i},dims=('j','i'))
    ds['lat'] = xr.DataArray(np.reshape( df['lat'].values, (jLen, iLen) ),\
                             coords={'j':j,'i':i},dims=('j','i'))
    ds['depth'] = xr.DataArray(np.reshape( df['depth'].values, (jLen, iLen) ),\
                               coords={'j':j,'i':i},dims=('j','i'))
    ds['depth'].attrs['units'] = 'm'
    ds = ds.set_coords(['lon','lat'])
    return ds

In [5]:
dsBathy = make_ij_dataset(bathyDf)
dsBathy['lon'] = lons360to180(dsBathy['lon'])
dsBathy['depthNaN'] = dsBathy.depth.where(dsBathy.depth < 0, drop=True)
dsBathy

Unique values:
lat: 321
lon: 343
check for completeness: 0


<xarray.Dataset>
Dimensions:   (i: 343, j: 321)
Coordinates:
  * i         (i) int64 0 1 2 3 4 5 6 7 8 ... 335 336 337 338 339 340 341 342
  * j         (j) int64 0 1 2 3 4 5 6 7 8 ... 313 314 315 316 317 318 319 320
    lon       (j, i) float64 -75.59 -75.57 -75.56 ... -69.93 -69.91 -69.89
    lat       (j, i) float64 41.8 41.8 41.8 41.8 41.8 ... 37.7 37.7 37.7 37.7
Data variables:
    depth     (j, i) float64 409.0 441.0 515.0 ... -4.105e+03 -4.109e+03
    depthNaN  (j, i) float64 nan nan nan ... -4.105e+03 -4.105e+03 -4.109e+03

## Save as netCDF file

In [11]:
ncDir = ('%s/ncFiles' % bathyDir)
if not os.path.isdir(ncDir):
    os.makedirs(ncDir)

In [12]:
dsBathy.to_netcdf('%s/bathymetry.nc' % ncDir)

## Some extra information

In [13]:
bathyArr = dsBathy.depthNaN.values.ravel()
bathyArr = bathyArr[~np.isnan(bathyArr)]
print("Max Depth is %0.3f"%(-1*np.min(bathyArr)))
print("Mean Depth is %0.3f"%(-1*np.mean(bathyArr)))

Max Depth is 4109.000
Mean Depth is 1047.598


<sub> Written by Escobar, I. 29 December 2020. Edited 04 Aug 2021 </sub>